This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card.

When SD card removed it will stream to UDP socket 9019 
(once an incoming UDP message is received on same port to 
192.168.4.1.)

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

(Should work on a car ride)

* Fix the humidity work
* Fix the initial logger plot
* Fix the analog reader rate
* missing date records: Rt0000B90Fd"2017-06-24T13:44:54.600"e000F423Fn00000682f00000682o000F423FA

* Apparently the nickel wire is no good.  Should use a 0.025 thermocouple
https://journals.ametsoc.org/doi/abs/10.1175/1520-0426%281987%29004%3C0383%3AANTT%3E2.0.CO%3B2



In [36]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [37]:
from BNO055_funcs import InitBNO055, readhexlifyBNO055, GetEulerAngles
print(InitBNO055())
from BNO055_funcs import uart2
print(uart2)

Temp25
UART(2, baudrate=115201, bits=8, parity=None, stop=1, tx=18, rx=2, rts=-1, cts=-1, timeout=0, timeout_char=1)


In [68]:
k = readhexlifyBNO055()
print(GetEulerAngles())

(10.03801, 48.03199, 355.0038)


In [352]:
%fetchfile --binary sd/LOG/016.TXT


Fetched 12562635=12562635 bytes from sd/LOG/016.TXT.
Saving file to '016.TXT'

In [331]:
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BNO055_funcs.py
%sendtofile --quiet --source NewDataLogger_funcs.py

Sent 55 lines (2040 bytes) to SI7021_funcs.py.
Sent 145 lines (4083 bytes) to OLED_grapher.py.
Sent 82 lines (2811 bytes) to BNO055_funcs.py.
Sent 105 lines (3788 bytes) to NewDataLogger_funcs.py.


In [308]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py
%sendtofile --quiet --source BlueFly_funcs.py

# this is broken
#%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BNO055_funcs.py
%sendtofile --quiet --source NewDataLogger_funcs.py

Sent 105 lines (3788 bytes) to NewDataLogger_funcs.py.


In [35]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [176]:
%sendtofile --quiet --source NewDataLogger_funcs.py
%sendtofile --quiet --source sdcard_funcs.py
%sendtofile --quiet --source UDPsocket_funcs.py


Sent 92 lines (2781 bytes) to NewDataLogger_funcs.py.
Sent 40 lines (1556 bytes) to sdcard_funcs.py.
Sent 37 lines (870 bytes) to UDPsocket_funcs.py.


In [162]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [36]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART, ADC
import os, time, math, ustruct
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from NewDataLogger_funcs import ConnectBluefly, readlogbluefly
from NewDataLogger_funcs import InitSI7021, readlogSI7021, calchumidtemps
from sdcard_funcs import ConnectSDcardFile
from BNO055_funcs import InitBNO055, readhexlifyBNO055, GetEulerAngles
from NewDataLogger_funcs import displayEulerAngles

# SD card uses sck=5, mosi=22, miso=27, cs=23
# OLED uses rst=16, scl=15, sda=4
# uartBlueFly = UART(1, baudrate=57600, rx=13, tx=17)
# Pin36 nickel input, Pin19 bridge resistor
# uartBNO055 = UART(2, baudrate=115200, rx=2, tx=18)

# interrupt PRG button used to cycle through modes
butt0 = Pin(0)  
butt0presses = 0
def butt0press(e):
    global butt0presses
    butt0presses += 1
butt0.irq(butt0press, butt0.IRQ_FALLING)

# input value from wire sensor (move to _funcs)
nickelwire = ADC(Pin(36))
nickelwire.atten(nickelwire.ATTN_11DB)
nickelwire.width(nickelwire.WIDTH_12BIT)
nickelbridge = Pin(19, Pin.OUT)
nickelbridge.value(0)

ConnectBluefly(oledshowfattext, vol=20)

# just do the blue spot detecting if a couple presses have happened after waiting for bluefly
if butt0presses == 2:  
    import hangspotmain  # goes straight to receiving and plotting the hangspot pos

sdmessage = ConnectSDcardFile()
oledshowfattext(sdmessage)
time.sleep_ms(2000)
from sdcard_funcs import sdfile
if sdfile is not None:
    dwrite, dflush = sdfile.write, sdfile.flush
else:
    from UDPsocket_funcs import dwrite, dflush, port
    oledshowfattext(["UDP", "port%d" % port])
    time.sleep_ms(2000)

k1 = InitSI7021(i2c)
k = InitBNO055()
oledshowfattext(["BNO055", k, "SI7021", k1])
time.sleep_ms(2000)

def adddisplayscrollgraph(v, tstamp, l1, l2):  # move into _funcs
    addscrollgraph(v, tstamp)
    plotscrollgraph(fbuff)
    fbuff.text(l1,0,0,1)
    fbuff.text(l2,0,8,1)

prevflushstamp = 0
prevoledstamp = 0
prevnickelwireread = -100
prevnickelwirestamp = 0
scrollinit()
gpsfixes = 0
prevbutt0 = butt0.value()
prevdisplaymode = 0
prsrec = 100000
while True:
    tstamp = time.ticks_ms()
    displaymode = butt0presses%5
    
    # should use mag = Pin(19, Pin.OUT) to tune it, but doesn't work
    nickelwireread = nickelwire.read()
    if abs(nickelwireread - prevnickelwireread) >= 4 and tstamp > prevnickelwirestamp + 25:
        dwrite("Nt{:08X}s{:06X}\n".format(tstamp, nickelwireread))
        prevnickelwireread = nickelwireread
        prevnickelwirestamp = tstamp
        
    readlogSI7021(i2c, dwrite)
    lbf = readlogbluefly(dwrite)
    bno055rec = readhexlifyBNO055()
    if bno055rec:
        dwrite(bno055rec)
        
    if tstamp > prevflushstamp + 2000:
        prevflushstamp = tstamp
        k = dflush()
        if isinstance(k, tuple):
            oledshowfattext([k[0][:8], k[0][8:], str(k[1])])  # write "UDP" to make it easier
            time.sleep_ms(2000)

    # all data has been collected.  Now display it
    if type(lbf) == int:
        prsrec = lbf
                
    if displaymode != prevdisplaymode:
        scrollinit()
        k = "display%d"%displaymode
        oledshowfattext(k)
        print(k)
        prevdisplaymode = displaymode
        
    if displaymode == 0:  # display nmea situation
        if type(lbf) == bytes:
            if lbf[0] == ord("V"):
                pass
            elif lbf[0] == ord("Q") and len(lbf) == 43:
                oledshowfattext([lbf[11:19], lbf[29:37], lbf[20:28], lbf[38:42]])
            else:
                oledshowfattext([lbf[:8], lbf[8:16], lbf[16:24], lbf[24:32]])
            #print(lbf)
            
    elif displaymode == 4:
        if bno055rec and tstamp > prevoledstamp + 50:
            displayEulerAngles(fbuff, *GetEulerAngles())
            oledshow()
            bno055rec = None
            prevoledstamp = tstamp

    elif tstamp > prevoledstamp + 200: # scrolling values
        if displaymode == 1:
            adddisplayscrollgraph(prsrec-100000, tstamp, "prs", str(prsrec))
        elif displaymode == 2:
            adddisplayscrollgraph(nickelwireread, tstamp, "nickel", str(nickelwireread))
        elif displaymode == 3:
            hs, ts = calchumidtemps()
            adddisplayscrollgraph(ts, tstamp, "temp", "%.2f"%ts)
        else:
            oledshowfattext(["mstamp", str(tstamp)])
        oledshow()
        prevoledstamp = tstamp


Sent 132 lines (4475 bytes) to main.py.


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [14]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f67265caf98, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [19]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 1, in <module>\r\n  File "OLED_driver.py", line 5, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [51]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.
